In [4]:
def modify_text_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    modified_content = content.replace('\n', ' ')

    modified_content = ' '.join(modified_content.split())

    with open(file_path, 'w') as file:
        file.write(modified_content)

file_path = 'text.txt'

modify_text_file(file_path)
